In [1]:
import numpy as np
import pandas as pd
import os 
import sys
import time
import matplotlib.pyplot as plt
from DataPreprocess import DataPreprocess

class mergeRegion:
    def __init__(self):
        self.path = os.getcwd()   # '\\region\\{}'.format(region)
        self.region_list = ['bucheon', 'bundang', 'busan', 'hongdae', 'jamsil', 'gangnam','incheon']
        self.path2 =os.path.abspath(os.path.join(self.path, "..", ".."))

    def makeDir(self):
        os.path.abspath(os.path.join(self.path, "..", ".."))
        self.newPath = os.path.join(f'{self.path2}\\MergeRegionData')
        if not os.path.exists(self.newPath):
            os.makedirs(os.path.join(f'{self.path2}\\MergeRegionData'))
        else:
            print('이미 존재합니다.')

    def delDir(self):
        self.newPath = os.path.join(f'{self.path2}\\MergeRegionData')
        # 'PreprocessData' 폴더가 존재하는지 확인하고 삭제
        if os.path.exists(self.newPath) and os.path.isdir(self.newPath):
            try:
                shutil.rmtree(self.newPath)
                print(f"'{self.newPath}' 폴더가 성공적으로 삭제되었습니다.")
            except Exception as e:
                print(f"폴더를 삭제하는 중 오류가 발생했습니다: {e}")
        else:
            print(f"'{self.newPath}' 폴더가 존재하지 않습니다.")

    def dataSet(self):

        self.preprocessPathList = []
        self.dataDict = {}
        for region in self.region_list:
            preprocessPath = os.path.join(f'{self.path2}\\region\\{region}\\PreprocessData\\{region}_merge.csv')
            self.preprocessPathList.append(preprocessPath)
            self.dataDict[region] = pd.read_csv(preprocessPath, encoding='utf_8', index_col=0)
            self.dataDict[region] = self.dataDict[region][~(self.dataDict[region]['ConsultTime'].isna())]

        self.fileList = self.preprocessPathList
        print('All files are added')
        return self.dataDict

    def regionCol(self):

        for region in self.region_list:
            df = self.dataDict[region]
            df['Region'] = f'{region}'

    def concatData(self):
        self.newPath = os.path.join(f'{self.path2}\\MergeRegionData')
        for i, region in enumerate(self.region_list):
            df = self.dataDict[region]
            if i == 0:
                concat_df = df
            else:
                concat_df = pd.concat([concat_df, df],axis = 0)
        self.concat_df = concat_df
        return self.concat_df

    def delZeroNaNCol(self, df):
        
        # 모든 원소가 NaN이거나 0인 열 삭제
        original_columns = df.columns  # 원본 열 이름 기억
        
        df_cleaned = df.dropna(axis=1, how='all')  # NaN 값이 모두인 열 삭제
        deleted_columns = original_columns.difference(df_cleaned.columns)  # 삭제된 열 찾기
        
        df_cleaned = df_cleaned.loc[:, (df_cleaned != 0).any(axis=0)]  # 0 값이 모두인 열 삭제
        deleted_columns = deleted_columns.union(original_columns.difference(df_cleaned.columns))  # 추가 삭제된 열 찾기
        
        print("\n처리된 데이터프레임:")
        print(df_cleaned)
        print("\n삭제된 열 이름:")
        print(deleted_columns)

 
        
    

In [3]:
# 병합 함수
def merge_rows(group):
    result = {'PatientID': group['PatientID'].iloc[0]}
    for col in group.columns:
        if col != 'PatientID':
            # 열 이름에 접미사 추가
            for i, value in enumerate(group[col], start=1):
                col_name = f"{col}_{i}"
                result[col_name] = value
    return result

# 최적화를 위한 데이터 처리
def process_and_merge(df, batch_size=1000):
    merged_results = []
    unique_ids = df['PatientID'].unique()
    
    for start in range(0, len(unique_ids), batch_size):
        end = start + batch_size
        batch_ids = unique_ids[start:end]
        batch_df = df[df['PatientID'].isin(batch_ids)]
        
        merged_results.extend([merge_rows(group) for _, group in batch_df.groupby('PatientID')])
    
    return pd.DataFrame(merged_results)


In [2]:
Merge = mergeRegion()
Merge.dataSet()

C:\Users\howsr\AppData\Local\Temp\ipykernel_8940\1831456504.py:42: DtypeWarning: Columns (28,54,62) have mixed types. Specify dtype option on import or set low_memory=False.
  self.dataDict[region] = pd.read_csv(preprocessPath, encoding='utf_8', index_col=0)
C:\Users\howsr\AppData\Local\Temp\ipykernel_8940\1831456504.py:42: DtypeWarning: Columns (4,28,56,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  self.dataDict[region] = pd.read_csv(preprocessPath, encoding='utf_8', index_col=0)
C:\Users\howsr\AppData\Local\Temp\ipykernel_8940\1831456504.py:42: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  self.dataDict[region] = pd.read_csv(preprocessPath, encoding='utf_8', index_col=0)
C:\Users\howsr\AppData\Local\Temp\ipykernel_8940\1831456504.py:42: DtypeWarning: Columns (28,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  self.dataDict[region] = pd.read_csv(preprocessPa

All files are added


{'bucheon':                  PatientID  ScheduleID          ConsultTime  ConsultNote  \
 MedicalRecordID                                                            
 876726.0                 9   1126286.0  2022-11-23 10:55:00          NaN   
 942106.0                 9   1195711.0  2023-02-16 09:45:00          NaN   
 943030.0                 9   1196745.0  2023-02-17 10:25:00          NaN   
 956320.0                 9   1210915.0  2023-03-07 19:20:00          NaN   
 956596.0                 9   1211226.0  2023-03-08 09:25:00          NaN   
 ...                    ...         ...                  ...          ...   
 1201055.0           243116   1490846.0  2024-05-14 15:45:00          NaN   
 1201001.0           243126   1490790.0  2024-05-14 14:45:00          NaN   
 1201040.0           243135   1490841.0  2024-05-14 15:35:00          NaN   
 1201076.0           243136   1490871.0  2024-05-14 16:00:00          NaN   
 1201146.0           243141   1490885.0  2024-05-14 17:25:00     

In [4]:
Merge.regionCol()

In [5]:
bundang = Merge.dataDict['bundang']

In [6]:
bundang['MedicineName'].gcount()

79585

In [6]:
df = Merge.concatData()

In [7]:
Merge.delZeroNaNCol(df)


처리된 데이터프레임:
                 PatientID  ScheduleID          ConsultTime ConsultNote  \
MedicalRecordID                                                           
876726.0                 9   1126286.0  2022-11-23 10:55:00         NaN   
942106.0                 9   1195711.0  2023-02-16 09:45:00         NaN   
943030.0                 9   1196745.0  2023-02-17 10:25:00         NaN   
956320.0                 9   1210915.0  2023-03-07 19:20:00         NaN   
956596.0                 9   1211226.0  2023-03-08 09:25:00         NaN   
...                    ...         ...                  ...         ...   
53615.0              21816     61480.0  2024-05-14 13:05:00         NaN   
53625.0              21821     61490.0  2024-05-14 13:05:00         NaN   
53661.0              21836     61545.0  2024-05-14 15:50:00         NaN   
53666.0              21840     61546.0  2024-05-14 16:15:00         NaN   
53676.0              21841     61556.0  2024-05-14 16:50:00         NaN   

           

In [8]:
group_sizes = df.groupby('PatientID').size()

In [9]:
# 그룹 크기가 1인 PatientID 필터링
single_patient_ids = group_sizes[group_sizes == 1].index
multiple_patient_ids1 = group_sizes[(group_sizes > 1) & (group_sizes <= 5)].index
multiple_patient_ids2 = group_sizes[(group_sizes > 5) & (group_sizes <= 9)].index
multiple_patient_ids3 = group_sizes[9 < group_sizes].index

# 필터링된 데이터 프레임
df_1 = df[df['PatientID'].isin(single_patient_ids)]
df_5 = df[df['PatientID'].isin(multiple_patient_ids1)]
df_10 = df[df['PatientID'].isin(multiple_patient_ids2)]
df_50 = df[df['PatientID'].isin(multiple_patient_ids3)]

In [67]:
A = [len(df_1) ,len(df_5), len(df_10),  len(df_50)]
print(A, sum(A))

[17698, 193119, 167370, 134404] 512591


In [56]:
# 데이터 병합
result_df50 = process_and_merge(df_50)

# 결과 확인
print("\nNew DataFrame with merged rows:")

new_columns =['PatientID']
columns = result_df50.columns
for num in range(1, 10):
    imsi_list = [column for column in columns if column[-2:] == '_'+str(num)]
    #print(imsi_list)
    new_columns = new_columns + imsi_list
for num in range(10, 49):
    imsi_list = [column for column in columns if column[-2:] == str(num)]
    new_columns = new_columns + imsi_list

result_df50 = result_df50[new_columns]
print(result_df50)
result_df50.to_csv(f'{Merge.newPath}\\platten_50.csv', index = False)


New DataFrame with merged rows:
       PatientID  ScheduleID_1        ConsultTime_1  ConsultNote_1  \
0              9     1126286.0  2022-11-23 10:55:00            NaN   
1             37     1208375.0  2023-03-06 16:10:00            NaN   
2             69     1143596.0  2022-12-10 11:15:00            NaN   
3             82     1141960.0  2022-12-02 10:25:00            NaN   
4            105     1216486.0  2023-03-10 16:30:00            NaN   
...          ...           ...                  ...            ...   
10575     296866     1774901.0  2023-09-21 10:05:00            NaN   
10576     297210     1776770.0  2023-09-19 12:10:00            NaN   
10577     297235     1777025.0  2023-09-18 11:30:00            NaN   
10578     298546     1819601.0  2023-12-18 10:40:00            NaN   
10579     304641     1844946.0  2024-02-01 16:25:00            NaN   

                                          ProgressNote_1  Summary_1  \
0      # 비만\n\n현재체중 : 65kg\n마지막 약 복용일자 : 복용 중\n포만감&식사..

In [57]:
# 데이터 병합
result_df10 = process_and_merge(df_10)

# 결과 확인
print("\nNew DataFrame with merged rows:")
columns = result_df10.columns
new_columns =['PatientID']

for num in range(1, 10):
    imsi_list = [column for column in columns if column[-2:] == '_'+str(num)]
    #print(imsi_list)
    new_columns = new_columns + imsi_list


result_df10 = result_df10[new_columns]
print(result_df10)
result_df10.to_csv(f'{Merge.newPath}\\platten_10.csv', index = False)


New DataFrame with merged rows:
       PatientID  ScheduleID_1        ConsultTime_1  ConsultNote_1  \
0             19     1134186.0  2022-11-28 17:45:00            NaN   
1             50     1139246.0  2022-12-01 14:05:00            NaN   
2             72     1242330.0  2023-04-12 14:10:00            NaN   
3            201     1210756.0  2023-03-07 18:50:00            NaN   
4            227     1190540.0  2023-02-08 19:15:00            NaN   
...          ...           ...                  ...            ...   
26200      17600       50060.0  2024-03-12 11:15:00            NaN   
26201      18510       52606.0  2024-03-23 11:15:00            NaN   
26202      19096       54246.0  2024-04-11 18:55:00            NaN   
26203      19791       56161.0  2024-04-11 14:30:00            NaN   
26204      20711       58556.0  2024-04-25 14:25:00            NaN   

                                          ProgressNote_1  Summary_1  \
0       \n# 비만\n\n현재체중 : 60kg\n마지막 약 복용일자 : 복용 중\n포만감..

In [58]:
# 데이터 병합
result_df5 = process_and_merge(df_5)

# 결과 확인
print("\nNew DataFrame with merged rows:")

columns = result_df5.columns
new_columns =['PatientID']
for num in range(1, 6):
    imsi_list = [column for column in columns if column[-2:] == '_'+str(num)]
    #print(imsi_list)
    new_columns = new_columns + imsi_list


result_df5 = result_df5[new_columns]
print(result_df5)
result_df5.to_csv(f'{Merge.newPath}\\platten_5.csv', index = False)


New DataFrame with merged rows:
       PatientID  ScheduleID_1        ConsultTime_1 ConsultNote_1  \
0             14     1309095.0  2023-06-29 14:50:00           NaN   
1             29     1117491.0  2022-11-11 14:40:00           NaN   
2             39     1142876.0  2022-12-02 16:05:00           NaN   
3             68     1117415.0  2022-11-11 11:40:00           NaN   
4            127     1128576.0  2022-11-24 15:30:00           NaN   
...          ...           ...                  ...           ...   
58764      21560       60700.0  2024-05-09 09:45:00           NaN   
58765      21711       61415.0  2024-05-13 19:05:00           NaN   
58766      21721       61281.0  2024-05-13 14:20:00           NaN   
58767      21780       61335.0  2024-05-13 16:05:00           NaN   
58768      21781       61326.0  2024-05-13 18:05:00           NaN   

                                          ProgressNote_1  Summary_1  \
0                                                    NaN        NaN

In [61]:
result_df1 = df_1
print(result_df1)

                 PatientID  ScheduleID          ConsultTime ConsultNote  \
MedicalRecordID                                                           
884214.0               183   1134009.0  2022-11-28 16:40:00         NaN   
1165140.0              211   1444451.0  2024-02-13 18:25:00         NaN   
1184781.0              217   1468981.0  2024-03-29 12:35:00         NaN   
892956.0               293   1143046.0  2022-12-02 17:00:00         NaN   
1197250.0              324   1485715.0  2024-05-03 11:35:00         NaN   
...                    ...         ...                  ...         ...   
53506.0              21785     61386.0  2024-05-13 17:25:00         NaN   
53620.0              21796     61391.0  2024-05-14 13:05:00         NaN   
53661.0              21836     61545.0  2024-05-14 15:50:00         NaN   
53666.0              21840     61546.0  2024-05-14 16:15:00         NaN   
53676.0              21841     61556.0  2024-05-14 16:50:00         NaN   

                        

In [52]:
cols = result_df1.columns.tolist()
new_cols = ['PatientID']+ [col + '_1' for col in cols[1:]]
new_cols

['PatientID',
 'ScheduleID_1',
 'ConsultTime_1',
 'ConsultNote_1',
 'ProgressNote_1',
 'Summary_1',
 'RequestI_1',
 'RequestII_1',
 'TreatmentCat_1',
 'Disclosure_1',
 'InsClaimAmt_1',
 'InsChargeAmt_1',
 'InsTotalAmt_1',
 'NonInsAmt_1',
 'AidAmt_1',
 'DisabledAidAmt_1',
 'HealthCareAmt_1',
 'DiscountAmt_1',
 'Bookmark_1',
 'InsuranceType_1',
 'Discarded_1',
 'DoctorID_x_1',
 'RefundAmt_1',
 'PayType_1',
 'ModTime_1',
 'MedicinePrice_1',
 'ArbitaryPrice_1',
 'VCode_1',
 'UnpaidCalFlag_1',
 'RefundOrder_1',
 'RefundDate_1',
 'DetailMemo_1',
 'TreatmentCode_1',
 'SourceProgram_1',
 'LossAmt_1',
 'ConResult_1',
 'TaxableAmt_1',
 'TDiscountAmt_1',
 'PregGrantAmt_1',
 'MedicalRecordID_ORG_1',
 'CancerNo_1',
 'ExceptClaim_1',
 'InfantFlag_1',
 'Ins100Amt_1',
 'LtClaimAmt_1',
 'LtChargeAmt_1',
 'MedicineName_1',
 'MedicineCode_1',
 'Memo_1',
 'MedicationID_1',
 'NoApps_1',
 'TotalApps_1',
 'NoDays_1',
 'Chup_1',
 'Pack_1',
 'Dose_1',
 'Price_1',
 'Additional_1',
 'Category1_1',
 'Category2_1'

In [54]:
result_df1.columns = new_cols
result_df1

,PatientID,ScheduleID_1,ConsultTime_1,ConsultNote_1,ProgressNote_1,Summary_1,RequestI_1,RequestII_1,TreatmentCat_1,Disclosure_1,...,SMM_1,BodyFat_1,VFA_1,WHR_1,CheckDate_1,CrTime_1,MaxVital_1,MinVital_1,Pulse_1,Region_1
MedicalRecordID,,,,,,,,,,,,,,,,,,,,,
884214.0,183,1134009.0,2022-11-28 16:40:00,NaN,# 비만\n\n현재체중 : 64kg\n마지막 약 복용일자 : 3개월전\n포만감&식사...,NaN,24.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bucheon
1165140.0,211,1444451.0,2024-02-13 18:25:00,NaN,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:168 / 73 / 경도비만(...,NaN,200.0,NaN,0.0,1.0,...,23.0,30.4,166.6,0.98,2024-02-13 18:03:00,2024-02-13 18:40:59,125.0,71.0,66.0,bucheon
1184781.0,217,1468981.0,2024-03-29 12:35:00,NaN,#비만(내원재진)\n\n*혈압: 112/69/81\n*체 중: (57.7 ) +\n...,NaN,118.0,NaN,0.0,1.0,...,22.4,16.0,71.4,0.86,2024-03-29 12:00:00,2024-03-29 12:25:37,112.0,69.0,81.0,bucheon
892956.0,293,1143046.0,2022-12-02 17:00:00,NaN,# 비만\n\n현재체중 : 52kg\n마지막 약 복용일자 : 1개월 이상\n포만감&...,NaN,25.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bucheon
1197250.0,324,1485715.0,2024-05-03 11:35:00,NaN,#비만(전화접수)\n\n현재체중 : 89\n포만감 또는 식욕조절 : 적당함\n불편사...,NaN,100.0,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bucheon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53506.0,21785,61386.0,2024-05-13 17:25:00,NaN,#비만(비대면초진)\n\n남자\n\nㆍ 키/체중/BMI:180 / 93 / 경도비만...,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,incheon
53620.0,21796,61391.0,2024-05-14 13:05:00,NaN,# 비만(비대면초진)\n\n· 키/체중/BMI : 155 / 51 / 21.23\n...,NaN,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,incheon
53661.0,21836,61545.0,2024-05-14 15:50:00,NaN,#비만(내원초진)\n\n여자\n\nㆍ 키/체중/BMI:162 / 73 / 경도비만(...,NaN,13.0,NaN,0.0,1.0,...,25.5,31.1,163.0,0.98,2024-05-14 15:05:00,2024-05-14 16:07:23,133.0,85.0,93.0,incheon


In [15]:
result_df5 = pd.read_csv(f'{Merge.newPath}\\platten_5.csv', encoding ='utf-8')
result_df10 = pd.read_csv(f'{Merge.newPath}\\platten_10.csv', encoding ='utf-8')
result_df50 = pd.read_csv(f'{Merge.newPath}\\platten_50.csv', encoding ='utf-8')

C:\Users\howsr\AppData\Local\Temp\ipykernel_12756\2976031202.py:1: DtypeWarning: Columns (3,27,61,62,253,254,285,319,511,512,523,543,577,578,769,770,801,835,1027,1028,1059,1093,1101,1285,1286) have mixed types. Specify dtype option on import or set low_memory=False.
  result_df5 = pd.read_csv(f'{Merge.newPath}\\platten_5.csv', encoding ='utf-8')
C:\Users\howsr\AppData\Local\Temp\ipykernel_12756\2976031202.py:2: DtypeWarning: Columns (27,55,61,253,254,285,311,319,511,512,543,577,578,769,770,801,835,1027,1028,1059,1093,1094,1285,1286,1293,1317,1351,1543,1544,1551,1575,1609,1801,1802,1833,1867,1875,2059,2060,2091,2125,2133,2317,2318) have mixed types. Specify dtype option on import or set low_memory=False.
  result_df10 = pd.read_csv(f'{Merge.newPath}\\platten_10.csv', encoding ='utf-8')
C:\Users\howsr\AppData\Local\Temp\ipykernel_12756\2976031202.py:3: DtypeWarning: Columns (61,62,69,253,254,311,319,327,511,512,577,585,769,770,801,835,843,1027,1028,1059,1093,1101,1285,1286,1317,1351,1359

In [55]:
byPatientID = pd.concat([result_df1, result_df5, result_df10, result_df50], axis = 0, join ='outer')

In [56]:
byPatientID.shape, df.shape,
#(17698, 193119, 196080, 134404, 512591)

((113252, 12385), (2, 12643))

In [22]:
len(set(df['PatientID'])), len(set(byPatientID['PatientID']))

(110381, 110381)

In [24]:
byPatientID.to_csv(f'{Merge.newPath}\\byPatientID.csv',index = False)

In [57]:
group_df = byPatientID.groupby('PatientID')
group_size = group_df.size()
li = group_size[group_size != 1].index.tolist()

In [59]:
li

[47,
 52,
 136,
 201,
 326,
 358,
 389,
 411,
 446,
 476,
 486,
 575,
 614,
 736,
 767,
 816,
 819,
 990,
 1026,
 1030,
 1117,
 1220,
 1326,
 1373,
 1439,
 1487,
 1525,
 1571,
 1615,
 1761,
 1861,
 2034,
 2107,
 2205,
 2299,
 2368,
 2466,
 2646,
 2991,
 3103,
 3105,
 3133,
 3186,
 3332,
 3334,
 3416,
 3577,
 3599,
 3652,
 3819,
 3856,
 3996,
 4026,
 4056,
 4259,
 4265,
 4349,
 4461,
 4470,
 4578,
 4887,
 5108,
 5170,
 5173,
 5229,
 5238,
 5302,
 5419,
 5502,
 5522,
 5586,
 5614,
 5661,
 5741,
 5752,
 5957,
 6067,
 6114,
 6251,
 6252,
 6310,
 6630,
 6647,
 6741,
 7057,
 7242,
 7255,
 7613,
 7644,
 7650,
 7721,
 7775,
 7780,
 7800,
 7860,
 7893,
 7916,
 8063,
 8099,
 8182,
 8235,
 8254,
 8286,
 8315,
 8376,
 8620,
 8684,
 8875,
 8951,
 9167,
 9305,
 9516,
 9590,
 9716,
 9731,
 9851,
 9893,
 9927,
 9959,
 9977,
 10073,
 10079,
 10085,
 10191,
 10244,
 10283,
 10331,
 10351,
 10506,
 10610,
 10674,
 10728,
 10897,
 10917,
 11092,
 11104,
 11185,
 11200,
 11322,
 11508,
 11548,
 11618,
 117

In [68]:
df = byPatientID
# PatientID가 리스트에 포함된 행들 선택
filtered_df = df[df['PatientID'].isin(li)]

In [72]:
pd.set_option('display.max_columns', 1000)

In [78]:
df[df['PatientID']==43996]

,PatientID,ScheduleID_1,ConsultTime_1,ConsultNote_1,ProgressNote_1,Summary_1,RequestI_1,RequestII_1,TreatmentCat_1,Disclosure_1,InsClaimAmt_1,InsChargeAmt_1,InsTotalAmt_1,NonInsAmt_1,AidAmt_1,DisabledAidAmt_1,HealthCareAmt_1,DiscountAmt_1,Bookmark_1,InsuranceType_1,Discarded_1,DoctorID_x_1,RefundAmt_1,PayType_1,ModTime_1,MedicinePrice_1,ArbitaryPrice_1,VCode_1,UnpaidCalFlag_1,RefundOrder_1,RefundDate_1,DetailMemo_1,TreatmentCode_1,SourceProgram_1,LossAmt_1,ConResult_1,TaxableAmt_1,TDiscountAmt_1,PregGrantAmt_1,MedicalRecordID_ORG_1,CancerNo_1,ExceptClaim_1,InfantFlag_1,Ins100Amt_1,LtClaimAmt_1,LtChargeAmt_1,MedicineName_1,MedicineCode_1,Memo_1,MedicationID_1,NoApps_1,TotalApps_1,NoDays_1,Chup_1,Pack_1,Dose_1,Price_1,Additional_1,Category1_1,Category2_1,Confirm_1,PrescMemo_1,PrescDeliverMemo_1,MedicationID_ORG_1,DoctorID_y_1,LicNo_1,OrderyCheck_1,Date_1,ReadingID_1,MeasureDate_1,ObesityDegree_1,BCM_1,BMR_1,BMC_1,ReadingID_ORG_1,Weight_1,Height_1,MonthAge_1,PWeight_1,PHeight_1,InterCellWater_1,ExtraCellWater_1,TotalBodyWater_1,ProteinMass_1,MineralMass_1,BodyFatMass_1,SoftLeanMass_1,FatFreeMass_1,Osseus_1,MaxInterCellWater_1,MinInterCellWater_1,MaxExtraCellWater_1,MinExtraCellWater_1,MaxProteinMass_1,MinProteinMass_1,MinBodyFatMass_1,MaxBodyFatMass_1,MinMineralMass_1,MaxMineralMass_1,PercentBodyWater_1,PercentProtein_1,PercentMineral_1,PercentBodyFat_1,IPercentBodyWater_1,IPercentProtein_1,IPercentMineral_1,IPercentBodyFat_1,ECW_TBW_1,ECF_TBF_1,EProtein_1,EMineral_1,EFat_1,EWeight_1,EMuscle_1,EBodyFat_1,EBodyWater_1,BalanceUpper_1,BalanceLower_1,BalanceWhole_1,StrengthUpper_1,StrengthLower_1,StrengthWhole_1,ESMM_1,EBMI_1,EPBFI_1,EWHR_1,EEdema_1,RightArmWeight_1,PRightArm_1,LeftArmWeight_1,PLeftArm_1,TrunkWeight_1,PTrunk_1,RightLegWeight_1,PRightLeg_1,LeftLegWeight_1,PLeftLeg_1,IRightArm_1,ILeftArm_1,ITrunk_1,IRightLeg_1,ILeftLeg_1,TargetWeight_1,WeightControl_1,FatControl_1,MuscleControl_1,FitnessScore_1,SkeletalBodyMass_1,IdealSMM_1,PSMM_1,neck_Circum_1,neck_CircumST_1,neck_CircumDF_1,neck_Muscle_1,neck_MuscleST_1,neck_MuscleDF_1,neck_Fat_1,neck_FatST_1,chest_Circum_1,chest_CircumST_1,chest_CircumDF_1,chest_Muscle_1,chest_MuscleST_1,chest_MuscleDF_1,chest_Fat_1,chest_FatST_1,abdomen_Circum_1,abdomen_CircumST_1,abdomen_CircumDF_1,abdomen_Muscle_1,abdomen_MuscleST_1,abdomen_MuscleDF_1,abdomen_Fat_1,abdomen_FatST_1,hip_Circum_1,hip_CircumST_1,hip_CircumDF_1,hip_Muscle_1,hip_MuscleST_1,hip_MuscleDF_1,hip_Fat_1,hip_FatST_1,Larm_Circum_1,Larm_CircumST_1,Larm_CircumDF_1,Larm_Muscle_1,Larm_MuscleST_1,Larm_MuscleDF_1,Larm_Fat_1,Larm_FatST_1,Rarm_Circum_1,Rarm_CircumST_1,Rarm_CircumDF_1,Rarm_Muscle_1,Rarm_MuscleST_1,Rarm_MuscleDF_1,Rarm_Fat_1,Rarm_FatST_1,Lleg_Circum_1,Lleg_CircumST_1,Lleg_CircumDF_1,Lleg_Muscle_1,Lleg_MuscleST_1,Lleg_MuscleDF_1,Lleg_Fat_1,Lleg_FatST_1,Rleg_Circum_1,Rleg_CircumST_1,Rleg_CircumDF_1,Rleg_Muscle_1,Rleg_MuscleST_1,Rleg_MuscleDF_1,Rleg_Fat_1,Rleg_FatST_1,1_RA_1,1_LA_1,1_TR_1,1_RL_1,1_LL_1,5_RA_1,5_LA_1,5_TR_1,5_RL_1,5_LL_1,50_RA_1,50_LA_1,50_TR_1,50_RL_1,50_LL_1,250_RA_1,250_LA_1,250_TR_1,250_RL_1,250_LL_1,500_RA_1,500_LA_1,500_TR_1,500_RL_1,500_LL_1,1000_RA_1,1000_LA_1,1000_TR_1,1000_RL_1,1000_LL_1,PBMF_1,PBF_1,PWHR_1,PVFA_1,BMI_1,SMM_1,BodyFat_1,VFA_1,WHR_1,CheckDate_1,CrTime_1,MaxVital_1,MinVital_1,Pulse_1,Region_1,ScheduleID_2,ConsultTime_2,ConsultNote_2,ProgressNote_2,Summary_2,RequestI_2,RequestII_2,TreatmentCat_2,Disclosure_2,InsClaimAmt_2,InsChargeAmt_2,InsTotalAmt_2,NonInsAmt_2,AidAmt_2,DisabledAidAmt_2,HealthCareAmt_2,DiscountAmt_2,Bookmark_2,InsuranceType_2,Discarded_2,DoctorID_x_2,RefundAmt_2,PayType_2,ModTime_2,MedicinePrice_2,ArbitaryPrice_2,VCode_2,UnpaidCalFlag_2,RefundOrder_2,RefundDate_2,DetailMemo_2,TreatmentCode_2,SourceProgram_2,LossAmt_2,ConResult_2,TaxableAmt_2,TDiscountAmt_2,PregGrantAmt_2,MedicalRecordID_ORG_2,CancerNo_2,ExceptClaim_2,InfantFlag_2,Ins100Amt_2,LtClaimAmt_2,LtChargeAmt_2,MedicineName_2,MedicineCode_2,Memo_2,MedicationID_2,NoApps_2,TotalApps_2,NoDays_2,Chup_2,Pack_2,Dose

In [79]:
df_unique_patient = df.drop_duplicates(subset=['PatientID'], keep='first')

In [80]:
len(df_unique_patient), len(df)

(110381, 113252)

In [ ]:
df_unique_patient.to_csv(f'{Merge.newPath}\\byPatientID.csv',index = False)